# Saliency Calculation using python

In this notebook we reimplement the saliency calculation as presented in [1].

During this course we will make 5 different implementations in order to explore different approaches and compare them in terms of simplicity and excecution time. The implementation go as follows:

1. Python and Numpy *(only for vector operations and eigen decomposition)*.
2. Numba and Numpy *(only for vector operations and eigen decomposition)*.
3. Numpy 
4. PyTorch
5. PyTorch on GPU


[1] G. Arvanitis, A. S. Lalos and K. Moustakas, "Robust and Fast 3-D Saliency Mapping for Industrial Modeling Applications," in IEEE Transactions on Industrial Informatics, vol. 17, no. 2, pp. 1307-1317, Feb. 2021, doi: 10.1109/TII.2020.3003455.

In [2]:
# library imports and initialization
import open3d as o3d
import numpy as np
data_path = './resources/'

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Data

In [3]:
# select the mesh you want to use
filename = '6.obj' 

# load the mesh using open3d
mesh = o3d.io.read_triangle_mesh(data_path + filename)
print(f"Triangle mesh with: {len(np.asarray(mesh.vertices))} vertices")

# Compute the vertex normals 
mesh.compute_vertex_normals()

# NOTE: draw_geometries will run a visualizer outside the jupyter notebook environment
o3d.visualization.draw_geometries([mesh])

Triangle mesh with: 33910 vertices


# Neighbor Search

The first thing to do when calculating saliency is to find the k nearest neighbors for each vertex of the mesh.

Open3d provides a structure to build a KDTree, that can be created directly from an open3d **Geometry**, such as a point cloud or a triangle mesh. 

So the steps we will follow are:
1. Create a KDTree Structure from the 3D mesh
2. Select a query point, find its KNN using the KDTree and visualize them
3. For each vertex of the mesh find its K nearest neighbors

### 1. Create the KDTree

In [49]:
knn_tree = o3d.geometry.KDTreeFlann(mesh)

### 2. Find and visualize the neighbor of a query point

In [4]:
query_idx = 12350
k = 50

# use the KDTree to find the knns
[k, idx, _] = knn_tree.search_knn_vector_3d(mesh.vertices[query_idx], k)
k, idx

(50,
 IntVector[12350, 12564, 12132, 12342, 12351, 12131, 12573, 12565, 12133, 12784, 11911, 12124, 12574, 12785, 11912, 12343, 12352, 11910, 12794, 13008, 12786, 11913, 12909, 12566, 12134, 11686, 12125, 11685, 12575, 11904, 12795, 13009, 11687, 12344, 12353, 11684, 13018, 13129, 12787, 11914, 13226, 13010, 11455, 11905, 11688, 12567, 11456, 12797, 12135, 12126])

In [5]:
# Create a point cloud using the selected points
vertices = np.asarray(mesh.vertices)
selected = vertices[idx]

pcd = o3d.utility.Vector3dVector(selected)
pcd = o3d.geometry.PointCloud(pcd)
pcd.paint_uniform_color((1.0, 0.0, 0.0))

# NOTE: Use +/- to increase/decrease point size
o3d.visualization.draw_geometries([mesh, pcd])

### 3. For each vertex of the mesh find its knns

As of open3d version 0.17.0 I did not find a way to run a knn search for all the vertices of the mesh at once. 
So we will have to use a for loop to parse all the vertices and find the knns for each one of them. 

In [6]:
k=5
knns=[]
vertices = np.asarray(mesh.vertices)

for v in vertices:

    [_, ids, _] = knn_tree.search_knn_vector_3d(v, k)
    knns.append(ids)

# Create a np.array to store this information
knns = np.stack(knns)
knns.shape

(33910, 5)

In [7]:
knns[:10, :]

array([[    0,     1,     2, 12555,    18],
       [    1, 12555,    37,     0, 12556],
       [    2,    33,    37,     0,    18],
       [    3,     5,    27,     4, 10973],
       [    4,     5, 10975,     3, 10972],
       [    5,     4,     3,    20, 10972],
       [    6,    10,     9,     7,     8],
       [    7,    39,    21,    19,     8],
       [    8,    31,     9,     7,    19],
       [    9,     8,    28,    29,    31]], dtype=int32)

## Let's wrap up everything into a function 
Create a function that will receive a o3d mesh with **N** vertices and the desired number of neighbors **k** as input
and return a np.array with shape [N, k] containing the indices of the neighboring vertices.

In [8]:
def knn_search(mesh, k):
    # create the kdTree
    knn_tree = o3d.geometry.KDTreeFlann(mesh)

    # list to store the neighbors
    knns=[]
    vertices = np.asarray(mesh.vertices)
    
    for v in vertices:
    
        [kt, ids, _] = knn_tree.search_knn_vector_3d(v, k)
        knns.append(ids)
    
    # Create a np.array to store this information
    knns = np.stack(knns)

    return knns

In [9]:
%timeit -n10 _=knn_search(mesh, 15)

126 ms ± 1.17 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Python Saliency calculation

We will start with a simple Saliency calculation using pure python and numpy. 

However, we set a limitation. Numpy will only be used to store the data and perform the eigen decomposition of the covariance matrix. 


The steps we will follow:
1. Find the indexes of the neighbors for each point
2. Get the normals of the neighbors to create the **N** matrix. Note that since we use a 3D mesh, we have access to the vertex normals.
3. Calculate the covariance matrix for each vertex.
4. Perform eigen decomposition and use the eigenvalus to compute the saliency.

In [10]:
normals = np.asarray(mesh.vertex_normals)
normals.shape

(33910, 3)

In [11]:
# 1. Find the indexes of the neighbors for each point
k=15
knns = knn_search(mesh, k)
knns.shape

(33910, 15)

In [12]:
# 2. Get the normals of the neighbors to create the N matrix. Note that since we use a 3D mesh, we have access to the vertex normals.
N = normals[knns]
N.shape

(33910, 15, 3)

In [13]:
from math import sqrt
# dummy for loop
saliency_vals = []

# parse each vertex of the mesh and use the normals of the neighbor to compute the covariance matrix
for n in N:

    # 3. Calculate the covariance matrix for each vertex
    
    # n.shape = 15, 3
    # cov = n.t(3, 15) x n(15, 3) --> cov(3, 3)
    cov = np.dot(n.transpose(), n)

    # 4. Perform eigen decomposition and use the eigenvalus to compute the saliency.
    eigenvalues, _ = np.linalg.eig(cov)

    eig1, eig2, eig3 = eigenvalues[0], eigenvalues[1], eigenvalues[2]
    saliency = 1 / sqrt(eig1 * eig1 + eig2 * eig2 + eig3 * eig3)

    saliency_vals.append(saliency)


saliency = np.stack(saliency_vals)

### Colorize the mesh with respect to the saliency values

In [14]:
# normalize saliency values
saliency_norm = (saliency - saliency.min()) / (saliency.max() - saliency.min())

color = np.zeros((len(saliency_vals), 3))
color[:, 0] = saliency_norm

mesh.vertex_colors = o3d.utility.Vector3dVector(color)

o3d.visualization.draw_geometries([mesh])

In [15]:
def visualize_saliency(mesh, saliency):
    color = np.zeros((len(saliency), 3))
    color[:, 0] = saliency
    mesh.vertex_colors = o3d.utility.Vector3dVector(color)

    o3d.visualization.draw_geometries([mesh])

## Let's wrap up everything into a function

In [61]:
# create a function that will get the mesh normals and the knn matrix and calculate the saliency
def saliency_python(normals, knns):
    
    N = normals[knns]

    saliency_vals = np.zeros((normals.shape[0]))

    # parse each vertex of the mesh and use the normals of the neighbor to compute the covariance matrix
    for i, n in enumerate(N):
            
        # n.shape = 15, 3
        # cov = n.t(3, 15) x n(15, 3) --> cov(3, 3)
        cov = np.dot(n.transpose(), n)
    
        eigenvalues, _ = np.linalg.eig(cov)
    
        eig1, eig2, eig3 = eigenvalues[0], eigenvalues[1], eigenvalues[2]
        saliency = 1 / sqrt(eig1 * eig1 + eig2 * eig2 + eig3 * eig3)
    
        saliency_vals[i] = saliency
    
    saliency_norm = (saliency_vals - saliency_vals.min()) / (saliency_vals.max() - saliency_vals.min())

    return saliency_norm

In [62]:
%timeit -n 10 _=saliency_python(normals, knns)

499 ms ± 6.96 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Numba Saliency Calculation

Numba uses a jit compiler to make the python code faster to excecute. 

The main constraint of numba is that is has to run only python and numpy related code and some options may still not be available. 
For example the advanced indexing $N = normals[knns]$ cannot be compiled with the jit compiler and has to be separated from the rest of the code. 

In [45]:
from numba import jit

@jit(nopython=True)
def saliency_numba_jit(N):
    
    saliency_vals = np.zeros((normals.shape[0]))

    # parse each vertex of the mesh and use the normals of the neighbor to compute the covariance matrix
    for i in range(len(N)):
        
        n = N[i]    
        # n.shape = 15, 3
        # cov = n.t(3, 15) x n(15, 3) --> cov(3, 3)
        cov = np.dot(n.transpose(), n)
    
        eigenvalues, _ = np.linalg.eig(cov)
        
        eig1, eig2, eig3 = eigenvalues[0], eigenvalues[1], eigenvalues[2]
        saliency = 1 / np.sqrt(eig1 * eig1 + eig2 * eig2 + eig3 * eig3)
    
        saliency_vals[i] = saliency
    
    return saliency_vals

In [46]:
# This advanced indexing cannot be compiled using numba
N = normals[knns]
saliency = saliency_numba_jit(N)
saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
visualize_saliency(mesh, saliency)

## Let's wrap up everything into a function

In [47]:
def saliency_numba(normals, knns):
    N = normals[knns]
    saliency = saliency_numba_jit(N)
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())

    return saliency

In [48]:
%timeit -n 10 _=saliency_numba(normals, knns)

83.5 ms ± 610 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Numpy saliency calculation

We will try to remove the outter loop all together and replace in with matrix operations.

Before we created the *N* matrix, with shape [N, k, 3], where N is the number of vertices and k the number of neighbors. 
Then we iterated through a for loop and did a matrix multiplication for each vertex. 
Now, we will try to do all these multiplications all together.

Our goal is to create a covariance matrix for each vertex. To store this matrix we will need a np.array of shape [N, 3, 3].

We will explore two ways of doing this, one with matrix multiplication and one using einsum.

In [22]:
N = normals[knns]
N.shape

(33910, 15, 3)

In [23]:
# Create the transpose matrix
Nt = N.transpose(0, 2, 1)
Nt.shape

(33910, 3, 15)

### Matrix Multiplication

In [24]:
cov = np.matmul(Nt, N)
cov.shape

(33910, 3, 3)

In [25]:
# Not that the operant '*' is not used for matrix multiplication in numpy
cov = Nt @ N
cov.shape

(33910, 3, 3)

### Einsum

In [26]:
cov = np.einsum('bik, bkj -> bij', Nt, N)
cov.shape

(33910, 3, 3)

## Eigen decomposition on the whole covariance matrix

This is done the same as before. Numpy knows that in order to decompose a matrix it has to be square, so when given an array of shape [N, 3, 3] it knows there are N square matrices of shape 3x3.

In [27]:
eigenvalues, _ = np.linalg.eig(cov) 
eigenvalues.shape

(33910, 3)

In [28]:
saliency = 1 / np.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
saliency.shape

(33910,)

In [29]:
saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
visualize_saliency(mesh, saliency)

## Let's wrap up everything into a function

In [30]:
def saliency_numpy(normals, knns):
    N = normals[knns]
    Nt = N.transpose(0, 2, 1)
    cov = Nt @ N
    eigenvalues, _ = np.linalg.eig(cov) 
    saliency = 1 / np.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
    return saliency

In [31]:
%timeit -n 10 _=saliency_numpy(normals, knns)

71.5 ms ± 1.46 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Torch saliency calculation

PyTorch is one of the main libraries used in Deep Learning. We will try to use pytorch instead of numpy. 

Instead on numpy arrays we use torch tensors. 

In [55]:
import torch 

# torch.tensor() can cast a numpy array to a torch tensor
normals_t = torch.tensor(normals)
knns_t = torch.tensor(knns).long()

Indexing in PyTorch is similar to numpy, so to create the **N** matrix:

In [56]:
N = normals_t[knns_t]

Next we need to create the inverse of the **N** matrix, **Nt**. *torch.permute* has a similar functionality as np.transpose.

In [57]:
Nt = N.permute(0, 2, 1)

The covariance matrix and the eigen decomposition is similar to numpy. Again we have to replace the numpy specific methods with the corresponding ones of pytorch. 

In [58]:
# Compute the covariance matrix
cov = Nt @ N
# Perform eigen decomposition
eigenvalues = torch.linalg.eigvals(cov).real # torch calculates complex eigenvalues so we have to keep the real part
# Calculate the saliency values
saliency = 1 / np.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
# Normalize the values in range(0,1) for visualization
saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
# Visualize the result
visualize_saliency(mesh, saliency)

## Let's wrap up everything into a function

In [59]:
def saliency_torch(normals, knns):
    normals_t = torch.tensor(normals)
    knns_t = torch.tensor(knns).long()
    
    N = normals_t[knns_t]
    Nt = N.permute(0, 2, 1)
    
    cov = Nt @ N
    eigenvalues = torch.linalg.eigvals(cov).real # torch calculates complex eigenvalues so we have to keep the real part
    
    saliency = 1 / torch.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
    
    return saliency

In [60]:
%timeit -n 10 _=saliency_torch(normals, knns)

19.8 ms ± 565 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Torch CUDA (GPU) saliency calculation

PyTorch gives you the ability to easily do the calculations on the GPU simply by transfering the data to the GPU.

Every tensor in PyTorch has an attribute called device. If the device is *cpu* then the tensor is located in the CPU. When the tensor is located on the GPU, the device name will not be *gpu*. Instead it will be something like *cuda:0*. CUDA is a library developed by NVIDIA to write code to run on NVIDIA gpus. Since only NVIDIA GPUs are supported by PyTorch it uses the keyword cuda instead of gpu. The number after the : indicates the number of the GPU, using zero indexing. 

It is important to note, that in order to do any operation between two tensors, they must be on the same device. 

In [66]:
normlas_t_gpu = normals_t.cuda()
knns_t_gpu = knns_t.cuda()

print(f"'normals_t' is located in the {normals_t.device} || 'normals_t_gpu' is located in the {normlas_t_gpu.device}")

'normals_t' is located in the cpu || 'normals_t_gpu' is located in the cuda:0


The rest of the PyTorch code remains the same no matter the device that we are using.

In [75]:
N = normlas_t_gpu[knns_t_gpu]
Nt = N.permute(0, 2, 1)

cov = Nt @ N        
eigenvalues = torch.linalg.eigvals(cov).real # torch calculates complex eigenvalues so we have to keep the real part

saliency = 1 / torch.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())

In we want to use a tensor outside PyTorch, for example turn it back to numpy, we have to first move it to the CPU.

In [76]:
saliency = saliency.cpu().numpy()
visualize_saliency(mesh, saliency)

## Let's wrap up everything into a function

In [77]:
def saliency_torch_cuda(normals, knns):
    normals_t = torch.tensor(normals).cuda()
    knns_t = torch.tensor(knns).long().cuda()
    
    N = normals_t[knns_t]
    Nt = N.permute(0, 2, 1)
    cov = Nt @ N
    
    eigenvalues = torch.linalg.eigvals(cov).real # torch calculates complex eigenvalues so we have to keep the real part
    saliency = 1 / torch.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
    
    return saliency.cpu().numpy()

In [78]:
%timeit -n 10 _=saliency_torch_cuda(normals, knns)

20.5 ms ± 1.2 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Transfer data to GPU only once

Copying the data of a tensor from the CPU to the GPU, and vise versa, takes some time. Usually, we use the GPU to do heavy operations. Because saliency calculation is not such a demanding task, moving the data back and forth between the CPU and the GPU takes a relatively considerable amount of time. 

To demonstrate the efficiency of the GPU, we will create a function that operates on data already on the GPU and will not ask it to return the data to the CPU.

In [80]:
def saliency_torch_cuda_once(normals_t, knns_t):
    N = normals_t[knns_t]
    Nt = N.permute(0, 2, 1)
    cov = Nt @ N
    
    eigenvalues = torch.linalg.eigvals(cov).real # torch calculates complex eigenvalues so we have to keep the real part
    saliency = 1 / torch.sqrt(eigenvalues[:, 0] * eigenvalues[:, 0] + eigenvalues[:, 1] * eigenvalues[:, 1] + eigenvalues[:, 2] * eigenvalues[:, 2])
    saliency = (saliency - saliency.min()) / (saliency.max() - saliency.min())
    
    return saliency

In [82]:
normals_t = torch.tensor(normals).cuda()
knns_t = torch.tensor(knns).long().cuda()

In [83]:
%timeit -n 10 _=saliency_torch_cuda_once(normals_t, knns_t)

18.4 ms ± 800 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Fast KNN computation

We notice that the computation time of saliency is around 20ms (This time may differ depending on the machine that you are using). 
However, if we go back and see the KNN calculation time, it is 125 ms, that is more than 6 times slower. 

Thankfully, Open3D provides a faster way to compute the KNNs for a whole mesh at once. 
The reason I did not introduce it earlier was because it operates on torch tensors and I wanted to make a smooth introduction to torch first. This method is a part of the Open3D ml library that is install along with Open3D.

In [85]:
import open3d.ml.torch as ml3d

We want to create a point cloud, containing the vertices of the original mesh, and store it as a torch.tensor.

In [88]:
vertices = np.asarray(mesh.vertices)
pc = torch.tensor(vertices)
print(pc.shape, type(pc))

torch.Size([33910, 3]) <class 'torch.Tensor'>


In [93]:
k=15
nsearch = ml3d.layers.KNNSearch()
# nsearch(target_points, query_points, number_of_neighbors)
ans = nsearch(pc, pc, k)

# this method returns among other information the neighbors_index
neighbors_index = ans.neighbors_index.reshape(pc.shape[0], -1) # Note that we have to reshape the result
neighbors_index.shape

torch.Size([33910, 15])

### Let's test that we get the expected result

In [97]:
saliency = saliency_torch(normals, neighbors_index)
visualize_saliency(mesh, saliency)

/tmp/ipykernel_87333/84531162.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  knns_t = torch.tensor(knns).long()


## Let's wrap up everything into a function 

In [103]:
def knn_search_fast(mesh, k): # use same inputs as before
    nsearch = ml3d.layers.KNNSearch()
    ans = nsearch(pc, pc, k)
    neighbors_index = ans.neighbors_index.reshape(pc.shape[0], -1)
    return neighbors_index

In [104]:
%timeit -n10 _=knn_search_fast(mesh, 15)

9.75 ms ± 573 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


# Homework:

## 1. Test the topology of the mesh to validate the KNN neighbors.
The KNN search may return neighbors that are near the query point in the euclidean space, but may be topologicaly very far. 
Let's take for example a human model that is scratching his head. A point on the head of the model may be close to a point of the arm in terms of euclidean distance but topologically the distance is very large. 
So in this task set a topological distance threshold and make sure all the neighbors are withing this limit. 